### **Paso 2.5 - Ingesta del archivo "results.json"**

Nos permite crear e indicar parámetros en tiempo de ejecución

<center><img src="https://i.postimg.cc/W1Y3PV9C/db72.png"></center>

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
v_data_source

Out[2]: 'Ergast'

In [ ]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
v_file_date

Out[4]: '2023-06-11'

In [ ]:
%run "../includes/configuration"

In [ ]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo JSON

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [ ]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
results_df = spark.read \
.schema(results_schema) \
.json(f"{raw_folder_path}/results.json")
#.json(f"{raw_folder_path}/{v_file_date}/results.json")

In [ ]:
results_df.show(truncate=False)

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|time       |milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|1       |18    |1       |1            |22    |1   |1       |1           |1            |10.0  |58  |1:34:50.616|5690616     |39        |2   |1:27.452      |218.3          |1       |
|2       |18    |2       |2            |3     |5   |2       |2           |2            |8.0   |58  |+5.478     |5696094     |41        |3   |1:27.739      |217.586        |1       |
|3       |18    |3       |3            |7     |7   |3       |3           |3            |6.

#### Paso 2 - Renombrar columnas y añadir columnas nuevas

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("data_source", lit(v_data_source)) \
                                    .withColumn("file_date", lit(v_file_date))

In [ ]:
results_with_columns_df.show(truncate=False)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+-----------+----------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|time       |milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|statusId|data_source|file_date |
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+-----------+----------+
|1        |18     |1        |1             |22    |1   |1       |1            |1             |10.0  |58  |1:34:50.616|5690616     |39         |2   |1:27.452        |218.3            |1       |Ergast     |2023-06-18|
|2        |18     |2        |2             |3     |5   |2       |2            |2             |8.0   |58  |+5.478     |5696094     |41   

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

In [ ]:
results_with_ingestion_date_df.show(truncate=False)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+-----------+----------+-----------------------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|time       |milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|statusId|data_source|file_date |ingestion_date         |
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+-----------+----------+-----------------------+
|1        |18     |1        |1             |22    |1   |1       |1            |1             |10.0  |58  |1:34:50.616|5690616     |39         |2   |1:27.452        |218.3            |1       |Ergast     |2023-06-18|2023-06-11 14:37:08.044|
|2        |18     |2        |2          

#### Paso 3 - Eliminar la columna no deseada

In [ ]:
from pyspark.sql.functions import col

In [ ]:
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

In [ ]:
results_final_df.show(truncate=False)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+-----------+----------+----------------------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|time       |milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|data_source|file_date |ingestion_date        |
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+-----------+----------+----------------------+
|1        |18     |1        |1             |22    |1   |1       |1            |1             |10.0  |58  |1:34:50.616|5690616     |39         |2   |1:27.452        |218.3            |Ergast     |2023-06-18|2023-06-11 14:37:18.33|
|2        |18     |2        |2             |3     |5   |2       |2            |2

#### Paso 4 - Escribir datos en el contenedor **processed** del ADLS como **parquet**

In [ ]:
# Escribimos el archivo con formato PARQUET en el contenedor "processed" y directorio "results"
# Particionamos el archivo según los valores de la columna "race_id"
# El parámetro "processed_folder_path" se encuentra en el notebook "configuration"
results_final_df.write.mode('overwrite').partitionBy('race_id').parquet(f"{processed_folder_path}/results")

In [ ]:
# Visualizamos los objetos que se encuentran en el directorio "results"
%fs
ls /mnt/formula1dl/processed/results

In [ ]:
dbutils.notebook.exit("Success")